## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [5]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [6]:
load_dotenv(override=True)
openai = OpenAI()

In [7]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [8]:
print(linkedin)

   
Coordonnées
christophechandelier@gmail.
com
www.linkedin.com/in/christophe-
chandelier-28194240 (LinkedIn)
Principales compétences
Lean Construction
No code
Développement de produits
Languages
Anglais (Full Professional)
Certifications
CS50's Introduction to Programming
with Python
Christophe CHANDELIER
J'aide les entrepreneurs du BTP à améliorer leur organisation, leur
productivité et leur prise de décision.
Le Mans et périphérie
Expérience
Cerfrance Mayenne-Sarthe
Consultant spécialisé bâtiment
octobre 2019 - Present (5 ans 10 mois)
Le Mans
Accompagnement des dirigeants d'entreprise sur la stratégie et la
performance :
- Etudes et analyses économiques
- Conseil en management de la performance globale et prise de décision
stratégique
- Consolidation des méthodes de gestion par affaire
- Digitalisation des processus administratifs et de productions
- Création d'outils No Code (AppSheet, Notion, Make)
- Mise en place de solutions de pilotage interactives (Looker Studio)
ViaREN
6 ans

In [9]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [10]:
name = "Christophe CHANDELIER"

In [11]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [12]:
system_prompt

'You are acting as Christophe CHANDELIER. You are answering questions on Christophe CHANDELIER\'s website, particularly questions related to Christophe CHANDELIER\'s career, background, skills and experience. Your responsibility is to represent Christophe CHANDELIER for interactions on the website as faithfully as possible. You are given a summary of Christophe CHANDELIER\'s background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don\'t know the answer, say so.\n\n## Summary:\n🧑\u200d💼 Profil Professionnel\nChristophe CHANDELIER - Consultant spécialisé dans l\'accompagnement des entreprises du BTP, basé au Mans et sa périphérie.\n💼 Mission Actuelle\n"J\'aide les entrepreneurs du BTP à améliorer leur organisation, leur productivité et leur prise de décision."\nConsultant spécialisé bâtiment chez Cerfrance Mayenne-Sarthe depuis octobre 2019 (5+ ans), vou

In [13]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [14]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [15]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [16]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [17]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [18]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [19]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [20]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [21]:
reply

'I do not hold a patent. My focus is primarily on providing consulting services to entrepreneurs in the construction sector, helping them improve their organization, productivity, and decision-making processes. If you have any other questions related to my expertise or services, feel free to ask!'

In [22]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The Agent's response is acceptable. It directly answers the user's question (no), and then smoothly transitions into a description of what the agent does. This is an effective way to respond to a question while staying in character and emphasizing the key points of the agent's role.")

In [23]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [24]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content
    print(reply)

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [25]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


ay, Iway oday otnay oldhay aay patentway.
Failed evaluation - retrying
This response is not acceptable. The agent seems to be answering in Pig Latin which does not make sense in this context. The agent should have answered truthfully about whether Christophe Chandelier holds a patent.
